# **Annual High Tide Flooding Outlook Notebook**

Contact User Services: tide.prediction@noaa.gov

Coastal Hazards Branch  
Oceanographic Division  
NOAA's Center for Operational Oceanographic Products and Services (CO-OPS)

Last updated on: 7/14/2025

## Overview
NOAA's Center for Operational Oceanographic Products and Services (CO-OPS) is the nation's source for [coastal inundation data](https://tidesandcurrents.noaa.gov/high-tide-flooding/annual-outlook.html) through its network of long-term water level gauges. In this notebook, we will review, plot, and examine CO-OPS's [Annual High Tide Flooding Outlook](https://tidesandcurrents.noaa.gov/est). This notebook will walk through how the Annual Outlook is calculated and how different graphics that are part of CO-OPS's Annual HTF product are produced.

## Prerequisites and running the Notebook
This notebook uses Python programming and users may run it either in **Google Colab** or **Jupyter Notebook**. To work offline, manage libraries and dependencies personally; using **Jupyter Notebook** is recommeended. If you want cloud-based instant access, seamless co-operation and have access to good internet connection, **Google Colab** is the best option.

This notebook is designed for all technical skill levels. To run the notebook, go to the [github](https://github.com/NOAA-CO-OPS/Coastal_Hazards_Example_Notebooks) page and click on the drop down menu under 'Code', then download the ZIP file. Extract the ZIP file and upload the AnnualHTFoutlook_Exploration_Notebook into [google colab](https://colab.research.google.com/). [Explore different station/s](https://tidesandcurrents.noaa.gov/est) and change the station variable and unit of interest in step 3. Lastly, click **Runtime** in the top menu and choose **Run All**. When the 'Choose Files' box in step 2 becomes active, click on it and upload the TR_local_projections.csv and Station_ID_lookup.csv. You will see a green check mark to the left of each cell as cells are sucessfully executed.

Users should expect to commit 1 to 2 hours to familiarize themselves with the notebook, upload data, run code and examine results.

## Learning Outcomes
In this notebook users will be able to:
  1.  Grab the annual outlook data from CO-OPS derived product API.
  2.  Explore results from historical and future-looking perspectives.

## Data Source and Documentations
Input and output documentation can be found in the [CO-OPS Data API](https://api.tidesandcurrents.noaa.gov/api/prod/), [CO-OPS Derived Product API v0.1](https://api.tidesandcurrents.noaa.gov/dpapi/prod/) and the [CO-OPS Metadata API](https://api.tidesandcurrents.noaa.gov/mdapi/prod/).

Standard templates are available within the [CO-OPS API URL Builder](https://tidesandcurrents.noaa.gov/api-helper/url-generator.html), but this notebook is designed to give you more flexiblity through building our own API template to generate queries from our Data API, MetaData API, and Derived Product API.

All data used for this notebook come from the CO-OPS API, except for the full spreadsheet of Annual Outlook results (2025_HTF_Annual_Outlook_full_data.csv) which includes the model fits, coefficients, p-values, and predictions for all models run for each station and a spreadsheet of past ENSO values (oni_annual_means_1950_2024.csv). The last is included so we can examine the influence of ENSO on the predicted HTF days.


## Software
This tutorial uses the following Python packages:
- datetime
- requests
- time
- pandas
- numpy
- matplotlib
- math
- plotly
-ast
- warnings

## **Step 1**
## Install and Load Packages
Lets check if you have these packages installed. If not, this cell will install them.

In [ ]:
# @title
import subprocess, sys
packages = ["datetime","requests","time","pandas","numpy","matplotlib","warnings","math","plotly","folium","branca","ast"]
for package in packages:
    try:
        __import__(package)
    except ImportError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', package])

Now we load our packages and assign them names to use throughout the notebook.

In [ ]:
# @title
import pandas as pd
import requests
import datetime as dt
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import warnings
import folium
from branca.colormap import LinearColormap
import ast
#comment the line below if using Jupyter Notebook
from google.colab import files
#this is to suppress user warnings
warnings.filterwarnings("ignore")

## **Step 2**
## Upload External Data
Here we will upload the full set of data for the lastest Annual Outlook run (2025_HTF_Annual_Outlook_full_data.csv) and the Annual Oceanic Nino Index (ONI) since 1950 (oni_annual_means_1950_2024.csv). These files can be found on the CO-OPS [Github](https://github.com/NOAA-CO-OPS/Annual-High-Tide-Flooding-HTF-Outlook) page in the Annual-High-Tide-Flooding-HTF-Outlook repository with this notebook.

In [ ]:
uploaded = files.upload()

## **Step 3**

### First we will visualize all the stations and create a [map](https://tidesandcurrents.noaa.gov/high-tide-flooding/annual-outlook.html?station=8575512) similar to the Annual Outlook map on CO-OPS website.

In [ ]:
met_year=2025

met_yr_url = 'https://api.tidesandcurrents.noaa.gov/dpapi/prod/webapi/htf/htf_met_year_annual_outlook.json?&met_year='+str(met_year)

#myurl = (server + product + ".json?station=" + station + "&affil=" + affil)

# Use "requests" to get the Monthly MSL Data
urlResponse = requests.get(met_yr_url)
content=urlResponse.json()

# Assign data to variables.
mydata = content['MetYearAnnualOutlook']

# Make it a Dataframe
annual_df = pd.DataFrame(mydata)

annual_df


,stnId,stnName,lat,lon,metYear,highConf,lowConf,projectDate,projectMethod
0,1611400,"Nawiliwili, HI",21.9544,-159.3561,2025,17,8,2025-06-24,Quadratic
1,1612340,"Honolulu, HI",21.303333,-157.86453,2025,18,7,2025-06-24,Quadratic
2,1612480,"Mokuoloe, HI",21.433056,-157.79,2025,21,9,2025-06-24,Quadratic
3,1615680,"Kahului, Kahului Harbor, HI",20.894945,-156.469,2025,20,10,2025-06-24,Quadratic
4,1617433,"Kawaihae, HI",20.0366,-155.8294,2025,40,21,2025-06-24,Quadratic
...,...,...,...,...,...,...,...,...,...
105,9449880,"Friday Harbor, WA",48.545277,-123.0125,2025,8,2,2025-06-24,Quadratic with ENSO Sensitivity
106,9751401,"Limetree Bay, VI",17.6947,-64.7538,2025,0,0,2025-06-24,No temporal trend with 19 year average
107,9751639,"Charlotte Amalie, VI",18.330584,-64.925804,2025,0,0,2025-06-24,No temporal trend with 19 year average
108,9755371,"San Juan, La Puntilla, San Juan Bay, PR",18.458944,-66.11642,2025,0,0,2025-06-24,No temporal trend with 19 year average


In [ ]:
#@title Updating coordinates to show the three west pacific islands with the rest of the stations

# Ensure lon column is numeric
annual_df['lon'] = pd.to_numeric(annual_df['lon'], errors='coerce')

station_ids_to_adjust = ['1630000', '1820000', '1890000']
annual_df.loc[annual_df['stnId'].isin(station_ids_to_adjust), 'lon'] -= 360


In [ ]:
# @title Station map of the 2025 Annual High Tide Flooding Outlook

# Define bins for colormap
bins = [0,5, 10, 15, 20, 25, 30]  # Adjust bins as needed

# Define colors for the bins (shades of red)
colors = ['white','#FFCCCC', '#FF9999', '#FF6666', '#FF3333', '#FF0000', '#CC0000', '#990000']

# Create a linear colormap with colors in bins
colormap = LinearColormap(colors=colors, index=bins, vmin=min(bins), vmax=max(bins))
# Create a smaller map centered around the top high confidence stations with a dark background
mymap = folium.Map(location=[37.0902, -125.7129], zoom_start=2, tiles='cartodb positron', width='75%', height='75%')

# Add markers for each station
for index, row in annual_df.iterrows():
    # Define popup content
    popup_content = f"Station ID: {row['stnId']}<br>"
    popup_content += f"High Confidence: {row['highConf']}<br>"
    popup_content += f"Low Confidence: {row['lowConf']}<br>"

    # Determine circle radius (keep it constant)
    radius = 5

    # Determine circle color based on highConf values
    color = colormap(row['highConf'])

    # Add circle marker for other stations
    folium.CircleMarker(location=[row['lat'], row['lon']], radius=radius, color=color, fill=True, fill_opacity=0.7,
                            popup=folium.Popup(popup_content, max_width=300)).add_to(mymap)

# Add the colormap to the map
colormap.caption = 'High Confidence'
mymap.add_child(colormap)

# Display the map
mymap


In [ ]:
# @title What stations have seen the largest change since the year 2000? Stations with over 10 days increase since 2000 are highlighted

#Read in the model output information
df = pd.read_csv(str(met_year)+'_HTF_Annual_Outlook_full_data.csv')
stnlist = annual_df[['stnId','lat','lon']]
df['stnID'] = df['stnID'].astype(int).astype(str)
df = pd.merge(df, stnlist, left_on='stnID', right_on='stnId')
df['mid_pred'] = (df['highConf']+df['lowConf'])/2
df['diff_2000'] = round(df['mid_pred'] - df['pred_2k'],2)

diff_2000 = df['diff_2000']

min_val=df['diff_2000'].min()
max_val=df['diff_2000'].max()

top_stations=df[df['diff_2000']>10]

# Define bins for colormap
bins = [0,5, 10, 15, 20, 25, 30]  # Adjust bins as needed

# Define colors for the bins (shades of red)
colors = ['#FFCCCC', '#FF9999', '#FF6666', '#FF3333', '#FF0000', '#CC0000', '#990000']
# Create a linear colormap from white to red
colormap = LinearColormap(colors=colors, index=bins,vmin=min(bins), vmax=max(bins))

# Create a smaller map centered around the top 10 high confidence stations with a dark background
mymap = folium.Map(location=[37.0902, -125.7129], zoom_start=2,tiles='cartodb positron', width='75%', height='75%')

# Add markers for each station
for index, row in df.iterrows():
    # Define popup content
    popup_content = f"Station ID: {row['stnID']}<br>"
    popup_content += f"Difference from 2000: {row['diff_2000']}<br>"

    # Define circle radius based on highConf values
    radius = 5  # Larger highConf values result in larger circles

    # Determine circle color based on highConf values
    color = colormap(row['diff_2000'])

 # Check if the station is in the top 10
    if row['stnID'] in top_stations['stnID'].tolist():
        # Add star marker for top 10 stations
        folium.Marker(location=[row['lat'], row['lon']], icon=folium.Icon(icon='star', color='red'),
                      popup=folium.Popup(popup_content, max_width=300)).add_to(mymap)
    else:
        # Add circle marker for other stations
        folium.CircleMarker(location=[row['lat'], row['lon']], radius=radius, color=color, fill=True, fill_opacity=0.7,
                            popup=folium.Popup(popup_content, max_width=300)).add_to(mymap)

# Add the colormap to the map
colormap.caption = 'Difference in number of HTF Days/Year'
mymap.add_child(colormap)

# Display the map
mymap


In [ ]:
# @title Difference in the number of days between trend+ENSO prediction and trend-only for 2025 outlook, positive values indicates more HTF days with ENSO

enso_diff = pd.DataFrame(columns=['diff_enso'])

for i in range(len(df)):
    station = df.loc[i,'stnID']
    station_df = df.loc[df['stnID']==station]

    if station_df['projectMethod'].iloc[0] == 'Linear With ENSO Sensitivity':
        x = station_df['lin_enso_pred'].iloc[0] - station_df['lin_pred'].iloc[0]
        enso_diff = pd.concat([enso_diff, pd.DataFrame({'diff_enso': [x]})], ignore_index=True)

    elif station_df['projectMethod'].iloc[0] == 'Quadratic With ENSO Sensitivity':
        x = station_df['quad_enso_pred'].iloc[0] - station_df['quad_pred'].iloc[0]
        enso_diff = pd.concat([enso_diff, pd.DataFrame({'diff_enso': [x]})], ignore_index=True)

    elif station_df['projectMethod'].iloc[0] == 'Linear':
        x = np.nan
        enso_diff = pd.concat([enso_diff, pd.DataFrame({'diff_enso': [x]})], ignore_index=True)

    elif station_df['projectMethod'].iloc[0] == 'Quadratic':
        x = np.nan
        enso_diff = pd.concat([enso_diff, pd.DataFrame({'diff_enso': [x]})], ignore_index=True)

    elif station_df['projectMethod'].iloc[0] == 'No Temporal Trend with Linear ENSO Sensitivity':
        x = station_df['no_t_lin_pred'].iloc[0] - station_df['avg19'].iloc[0]
        enso_diff = pd.concat([enso_diff, pd.DataFrame({'diff_enso': [x]})], ignore_index=True)

    elif station_df['projectMethod'].iloc[0] == 'No Temporal Trend with Quadratic ENSO Sensitivity':
        x = station_df['no_t_quad_pred'].iloc[0] - station_df['avg19'].iloc[0]
        enso_diff = pd.concat([enso_diff, pd.DataFrame({'diff_enso': [x]})], ignore_index=True)

    else:
        x = np.nan
        enso_diff = pd.concat([enso_diff, pd.DataFrame({'diff_enso': [x]})], ignore_index=True)

df['diff_enso'] = round(enso_diff['diff_enso'],2)

df_enso = df.dropna(subset=['diff_enso'])

df_enso = df_enso.reset_index(drop=True)

min_val=df['diff_enso'].min()
max_val=df['diff_enso'].max()

bins = [-5, -4, -3, -2, -1, 0, 1, 2, 3, 4, 5]  # Adjust bins as needed

# Define colors for the bins (blue to white to red)
colors = ['#0000FF', '#3399FF', '#66CCFF', '#99CCFF', '#E0E0FF', 'white', '#FF9999', '#FF6666', '#FF3333', '#FF6600', '#FF0000']
colormap = LinearColormap(colors=colors, index=bins, vmin=min(bins), vmax=max(bins))

# Create a map centered around the continental US
mymap = folium.Map(location=[37.0902, -125.7129], zoom_start=2,tiles='cartodb positron', width='75%', height='75%')

# Add markers for each station
for index, row in df_enso.iterrows():
    # Define popup content
    popup_content = f"Station ID: {row['stnID']}<br>"
    popup_content += f"Difference (num days): {row['diff_enso']}<br>"
  #  popup_content += f"Low Confidence: {row['lowConf']}<br>"

    # Define circle radius based on highConf values
    radius = 5  # Larger highConf values result in larger circles

    # Determine circle color based on highConf values
    color = colormap(row['diff_enso'])

    # Add circle marker to the map with popup
    folium.CircleMarker(location=[row['lat'], row['lon']], radius=radius, color=color, fill=True, fill_opacity=0.7,
                        popup=folium.Popup(popup_content, max_width=300)).add_to(mymap)

# Add the colormap to the map
colormap.caption = 'Difference in number of High Tide Flood Days'
mymap.add_child(colormap)

# Display the map
mymap

## **Step 4**

### From the maps above, pick a station of interest to do a deeper dive into the historical flooding and out to decadal projections.

In [ ]:
station_id = '9435380'

#This is the 2025 average predicted ONI from all models
predicted_oni = -0.2

## **Step 5**

### The Annual Outlook is based on fitting historical data to different trend fits, both with and without senstivity from the El Nino-Southern Oscillation (ENSO). Here we will plot our historic data with the chosen method of prediction for that station. If the chosen method has ENSO sensitivity, we will also plot the trend-only prediction to provide insight on the effect that ENSO has on the prediction.

In [ ]:
# @title
# functions
def quadratic_enso(x, z):
    return (a * x ** 2) + (b * x * z) + (c * x) + (d * z ** 2) + (e * z) + f
def quadratic(x):
    return (a * x ** 2) + (c * x) + f
def linear_enso(x,z):
    return (a * x) + (b*z) + c
def linear(x):
    return (a * x) + c
def no_t_lin_enso(z):
    return (a * z) + b
def no_t_quad_enso(z):
    return (a * z **2) + (b * z) + c

#read in enso info
enso_oni = pd.read_csv('oni_annual_means_1950_2024.csv')

forecast_avg = predicted_oni
enso_oni.loc[len(enso_oni)] = [int(met_year), np.nan, forecast_avg, np.nan]
enso_oni = enso_oni['ONI Met Year'].groupby(enso_oni['Year']).mean()

#getting historical flood counts from API
product = 'htf_met_year_annual'

server = "https://api.tidesandcurrents.noaa.gov/dpapi/prod/webapi/htf/"

myurl = (server + product +'.json?station='+station_id)

urlResponse = requests.get(myurl)

content=urlResponse.json()

mydata = content['MetYearAnnualFloodCount']

# Make it a Dataframe
obs_df = pd.DataFrame(mydata)
obs_df=obs_df[obs_df['metYear']>1949]
obs_df=obs_df[obs_df['metYear']<met_year]

obs = obs_df.dropna(subset=['minCount'])
obs['HTF']='days per year'
data_start_year = obs['metYear'].iloc[0]

################################################################################

station_df = df.loc[df['stnID']==station_id]

station_method = station_df['projectMethod']

#for stations with a project method with ENSO sensitivity, we also want to grab the time-based method as well
if station_df['projectMethod'].iloc[0] == 'Linear With ENSO Sensitivity':
        trend_only = station_df[['stnID','stnName','lin_pred','lin_rmse']]
        trend_only['Low_Pred'] = trend_only['lin_pred'] - trend_only['lin_rmse']
        trend_only['High_Pred'] = trend_only['lin_pred'] + trend_only['lin_rmse']

elif station_df['projectMethod'].iloc[0] == 'Quadratic With ENSO Sensitivity':
        trend_only = station_df[['stnID','stnName','quad_pred','quad_rmse']]
        trend_only['Low_Pred'] = trend_only['quad_pred'] - trend_only['quad_rmse']
        trend_only['High_Pred'] = trend_only['quad_pred'] + trend_only['quad_rmse']

elif station_df['projectMethod'].iloc[0] in ['No Temporal Trend with Linear ENSO Sensitivity', 'No Temporal Trend with Quadratic ENSO Sensitivity']:
        trend_only = station_df[['stnID','stnName','avg19','stdev19']]
        trend_only['Low_Pred'] = trend_only['avg19'] - trend_only['stdev19']
        if trend_only['Low_Pred'].iloc[0] < 0:
            trend_only['Low_Pred'] = 0
        trend_only['High_Pred'] = trend_only['avg19'] + trend_only['stdev19']

predict_df = station_df[['stnID','stnName','lowConf','highConf']]
predict_df.rename(columns={'lowConf':'Low_Pred','highConf':'High_Pred'},inplace=True)

#for plotting outlook with historical data
metYear=int(met_year)-0.25
nextYear=int(met_year)+0.35

try:
        trend_only['metYear']=metYear
        trend_only['nextYear']=nextYear

except NameError:
        print('Not an ENSO sensitive station')

predict_df['metYear']=metYear
predict_df['nextYear']=nextYear

    #######################################################################################

#getting trend formulas
if station_df['projectMethod'].iloc[0] == 'Linear With ENSO Sensitivity':
        station_formula = station_df['lin_enso_fit'].iloc[0]
        station_trend = station_df['lin_fit'].iloc[0]
        coefficients = ast.literal_eval(station_formula)
        coefficients_trend = ast.literal_eval(station_trend)
elif station_df['projectMethod'].iloc[0] == 'Quadratic With ENSO Sensitivity':
        station_formula = station_df['quad_enso_fit'].iloc[0]
        station_trend = station_df['quad_fit'].iloc[0]
        coefficients = ast.literal_eval(station_formula)
        coefficients_trend = ast.literal_eval(station_trend)
elif station_df['projectMethod'].iloc[0] == 'Linear':
        station_formula = station_df['lin_fit'].iloc[0]
        coefficients = ast.literal_eval(station_formula)
elif station_df['projectMethod'].iloc[0] == 'Quadratic':
        station_formula = station_df['quad_fit'].iloc[0]
        coefficients = ast.literal_eval(station_formula)
elif station_df['projectMethod'].iloc[0] == 'No Temporal Trend with Linear ENSO Sensitivity':
        station_formula = station_df['no_t_lin_fit'].iloc[0]
        station_trend = station_df['avg19'].iloc[0]
        coefficients = ast.literal_eval(station_formula)
        coefficients_avg = station_trend
elif station_df['projectMethod'].iloc[0] == 'No Temporal Trend with Quadratic ENSO Sensitivity':
        station_formula = station_df['no_t_quad_fit'].iloc[0]
        station_trend = station_df['avg19'].iloc[0]
        coefficients = ast.literal_eval(station_formula)
        coefficients_avg = station_trend
else:
        station_formula = station_df['avg19']
        coefficients_avg = station_formula

    #######################################################################################

    #calculating trend lines
data_length = int(met_year)-1949
x_vals = enso_oni.index
z = 0

        # Extract coefficients
if station_df['projectMethod'].iloc[0] == 'Quadratic With ENSO Sensitivity':
        a = coefficients['np.power(x, 2)']
        b = coefficients['x:z']
        c = coefficients['x']
        d = coefficients['np.power(z, 2)']
        e = coefficients['z']
        f = coefficients['Intercept']
        y_vals = quadratic_enso(x_vals, z)
        y_vals_trend = quadratic_enso(x_vals,enso_oni)
        other_method = 'Quadratic'
elif station_df['projectMethod'].iloc[0] =='Quadratic':
        a = coefficients['np.power(x, 2)']
        c = coefficients['x']
        f = coefficients['Intercept']
        y_vals_trend = quadratic(x_vals)
elif station_df['projectMethod'].iloc[0] == 'Linear With ENSO Sensitivity':
        a = coefficients['x']
        b = coefficients['z']
        c = coefficients['Intercept']
        y_vals = linear_enso(x_vals, z)
        y_vals_trend = linear_enso(x_vals,enso_oni)
        other_method = 'Linear'
elif station_df['projectMethod'].iloc[0] == 'Linear':
        a = coefficients['x']
        c = coefficients['Intercept']
        y_vals_trend = linear(x_vals)
elif station_df['projectMethod'].iloc[0] == 'No Temporal Trend with Linear ENSO Sensitivity':
        a = coefficients['z']
        b = coefficients['Intercept']
        y_vals_trend = no_t_lin_enso(enso_oni)
        y_vals = [coefficients_avg]*data_length
        other_method = '19-yr Average'
elif station_df['projectMethod'].iloc[0] == 'No Temporal Trend with Quadratic ENSO Sensitivity':
        a = coefficients['np.power(z, 2)']
        b = coefficients['z']
        c = coefficients['Intercept']
        y_vals_trend = no_t_quad_enso(enso_oni)
        y_vals = [coefficients_avg]*data_length
        other_method = '19-yr Average'
else:
        y_vals_trend = [coefficients_avg.iloc[0]]*data_length

if station_id == '1820000':
        max_y_lim = round(obs_df['minCount'].max()/5)*5+15
else:
        max_y_lim = round(obs_df['minCount'].max()/5)*5+10

    #######################################################################################

In [ ]:
# @title

# Create the figure and configure bar width
fig = go.Figure()
bar_width = 0.7

# Add bar trace for Historical HTF days
fig.add_trace(go.Bar(
    x=obs['metYear'],
    y=obs['minCount'],
    marker_color='lightskyblue',
    width=bar_width,
    name='Historical HTF days'
))

# Check station project method
if station_df['projectMethod'].iloc[0] in ['Linear With ENSO Sensitivity', 'Quadratic With ENSO Sensitivity',
                                           'No Temporal Trend with Linear ENSO Sensitivity',
                                           'No Temporal Trend with Quadratic ENSO Sensitivity']:
    # Add fill-between trace for other_method

    # Add bar trace for Historical HTF days

    fig.add_trace(go.Bar(
        x=[metYear],
        y=trend_only['High_Pred'],
        marker_color='black',
        width=bar_width,
        name=other_method,
    ))
    fig.add_trace(go.Bar(
        x=[metYear],
        y=trend_only['Low_Pred'],
        marker_color='white',
        width=bar_width,
        showlegend=False,
    ))

    fig.add_trace(go.Bar(
        x=[metYear+1],
        y=predict_df['High_Pred'],
        marker_color='red',
        width=bar_width,
        name=station_df['projectMethod'].iloc[0],
    ))
    fig.add_trace(go.Bar(
        x=[metYear+1],
        y=predict_df['Low_Pred'],
        marker_color='white',
        width=bar_width,
        showlegend=False,
    ))

    # Add line plot for x_vals and y_vals
    fig.add_trace(go.Scatter(
        x=x_vals,
        y=y_vals,
        mode='lines',
        line=dict(color='black', width=2, dash='dash'),
        showlegend=False,  # Exclude from legend
    ))

    # Add line plot for x_vals and y_vals_trend
    fig.add_trace(go.Scatter(
        x=x_vals,
        y=y_vals_trend,
        mode='lines',
        line=dict(color='red', width=2, dash='dash'),
        showlegend=False,  # Exclude from legend
    ))

else:
    # Add fill-between trace for station's project method
    fig.add_trace(go.Bar(
        x=predict_df['metYear'],
        y=predict_df['High_Pred'],
        marker_color='black',
        width=bar_width,
        name=station_df['projectMethod'].iloc[0],
    ))
    fig.add_trace(go.Bar(
        x=predict_df['metYear'],
        y=predict_df['Low_Pred'],
        marker_color='white',
        width=bar_width,
        showlegend=False,
    ))

    # Add line plot for x_vals and y_vals_trend
    fig.add_trace(go.Scatter(
        x=x_vals,
        y=y_vals_trend,
        mode='lines',
        line=dict(color='black', width=2, dash='dash'),
        showlegend=False,  # Exclude from legend
    ))

# Update layout with titles, axes labels, and legends
fig.update_layout(
    title='2024-25 Annual HTF Projection with Historical Contexte<br>' + obs_df.iloc[0]['stnId'] + ' ' + obs_df.iloc[0]['stnName'],
    xaxis=dict(title='Years', range=[data_start_year - 2, 2026]),
    yaxis=dict(title='Flood Count (days/yr)', range=[0, max_y_lim]),
    legend=dict(x=0, y=1.0, traceorder='reversed', title='Legend'),
    barmode='overlay',  # Overlay bars on top of each other
    bargap=0.1,  # Small gap between bars of adjacent location coordinates
    bargroupgap=0.1,  # Medium gap between bars of the same location coordinate,
    plot_bgcolor='rgba(255, 255, 255, 1)',  # White gridded background
    paper_bgcolor='rgba(255, 255, 255, 1)',
    width=1200,  # Set width of the plot
    height=800,
    title_x=0.5# White gridded background
)

# Show the plot
fig.show()


In [ ]:
# @title Focusing on the variability of recent years

# Create the figure and configure bar width
fig = go.Figure()
bar_width = 0.6

obs_recent = obs[obs['metYear']>2019]
# Add bar trace for Historical HTF days
fig.add_trace(go.Bar(
    x=obs_recent['metYear'],
    y=obs_recent['minCount'],
    marker_color='blue',
    width=bar_width,
    name='Observed Flood Days in Met Year'
))

# Add fill-between trace for station's project method
fig.add_trace(go.Bar(
        x=[met_year],
        y=predict_df['High_Pred'],
        marker_color='red',
        opacity=0.5,
        width=bar_width,
        name='Upper Range of Projected Flood Days in Met Year',
    ))
fig.add_trace(go.Bar(
        x=[met_year],
        y=predict_df['Low_Pred'],
        marker_color='red',
        width=bar_width,
        name='Lower Range of Projected Flood Days in Met Year',
    ))

# Update layout with titles, axes labels, and legends
fig.update_layout(
    title='Observed / Projected HTF Days<br>' + obs_df.iloc[0]['stnId'] + ' ' + obs_df.iloc[0]['stnName'],
    xaxis=dict(title='Years', range=[met_year-5, met_year+0.5]),
    yaxis=dict(title='Flood Count (days/yr)', range=[0, max_y_lim]),
    legend=dict(x=0, y=1.0, traceorder='reversed', title='Legend'),
    barmode='overlay',  # Overlay bars on top of each other
    bargap=0.1,  # Small gap between bars of adjacent location coordinates
    bargroupgap=0.1,  # Medium gap between bars of the same location coordinate,
    plot_bgcolor='rgba(255, 255, 255, 1)',  # White gridded background
    paper_bgcolor='rgba(255, 255, 255, 1)',
    width=1200,  # Set width of the plot
    height=600,
    title_x=0.5# White gridded background
)

# Show the plot
fig.show()


## **Step 5**

#Change the cell below to select an NOS flooding threshold

# Options are:


*   NOS Minor
*   NOS Moderate
*   NOS Major








In [ ]:
flood_threshold = 'minor'

In [ ]:
# @title Decadal Plot

sea_level_rise_scenarios = ['low', 'intLow', 'intermediate', 'intHigh', 'high']
# Initialize observed data container
avg_flood_days_per_decade = None

# Map flood_threshold to API count field names
api_count_col_map = {
    'minor': 'minCount',
    'moderate': 'modCount',
    'major': 'majCount'
}
obs_count_col = api_count_col_map.get(flood_threshold, 'minCount')

if flood_threshold in ['moderate', 'major']:
    # --- Pull observed met-year annual data only for moderate or major ---
    obs_url = f"https://api.tidesandcurrents.noaa.gov/dpapi/prod/webapi/htf/htf_met_year_annual.json?station={station_id}"
    obs_response = requests.get(obs_url)
    obs_content = obs_response.json()

    obs_df = pd.DataFrame(obs_content['MetYearAnnualFloodCount'])
    obs_df['metYear'] = obs_df['metYear'].astype(int)
    obs_df[obs_count_col] = obs_df[obs_count_col].astype(float)

    obs_decade = obs_df[(obs_df['metYear'] >= 1991) & (obs_df['metYear'] <= 2020)].copy()

    obs_decade['decade'] = (obs_decade['metYear'] // 10 + 1) * 10
    obs_decade.loc[obs_decade['metYear'] == 2000, 'decade'] = 2000
    obs_decade.loc[obs_decade['metYear'] == 2010, 'decade'] = 2010
    obs_decade.loc[obs_decade['metYear'] == 2020, 'decade'] = 2020

    avg_flood_days_per_decade = round(obs_decade.groupby('decade')[obs_count_col].mean().reset_index(), 0)

else:
    obs_decade = obs[(obs['metYear'] >= 1991) & (obs['metYear'] <= 2020)].copy()

    # Your original decade calculation to align with observed bars:
    obs_decade['decade'] = (obs_decade['metYear'] // 10 + 1) * 10
    obs_decade.loc[obs_decade['metYear'] == 2000, 'decade'] = 2000
    obs_decade.loc[obs_decade['metYear'] == 2010, 'decade'] = 2010
    obs_decade.loc[obs_decade['metYear'] == 2020, 'decade'] = 2020

    avg_flood_days_per_decade = round(obs_decade.groupby('decade')['minCount'].mean().reset_index(), 0)

# --- API request and projection data ---
url = 'https://api.tidesandcurrents.noaa.gov/dpapi/prod/webapi/htf/htf_projection_decadal.json?'
response = requests.get(url)
content = response.json()

# Extract list for chosen flood_threshold
decadal_list = []
for entry in content['DecadalProjection']:
    if flood_threshold in entry:
        decadal_list.extend(entry[flood_threshold])

decadal_proj = pd.DataFrame(decadal_list)
station_proj_df = decadal_proj[decadal_proj['stnId'] == station_id].copy()

# Keep only decades up to 2050 for projection plotting
station_proj_df = station_proj_df[station_proj_df['decade'] <= 2050]

scenario_colors = {
    'low': 'black',
    'intLow': 'purple',
    'intermediate': 'red',
    'intHigh': 'orange',
    'high': 'yellow'
}

fig = go.Figure()

obs_count_col = 'minCount'  # default to minor
if flood_threshold == 'moderate':
    obs_count_col = 'modCount'
elif flood_threshold == 'major':
    obs_count_col = 'majCount'

# Plot observed HTF days as bars
fig.add_trace(go.Bar(
    x=avg_flood_days_per_decade['decade'],
    y=avg_flood_days_per_decade[obs_count_col],
    marker_color='blue',
    width=4,
    name='Average annual observed flood days per decade'
))

# Plot projections lines for each scenario
for scenario in sea_level_rise_scenarios:
    if scenario in station_proj_df.columns:
        fig.add_trace(go.Scatter(
            x=station_proj_df['decade'],
            y=station_proj_df[scenario],
            mode='lines+markers',
            line=dict(color=scenario_colors[scenario], width=3),
            marker=dict(size=6),
            name=f'{flood_threshold.capitalize()} HTF days ({scenario})'
        ))

fig.update_layout(
    title=obs_df.iloc[0]['stnId'] + ' ' + obs_df.iloc[0]['stnName']+'<br> NOS ' + flood_threshold+' threshold',
    xaxis=dict(title='Decade', range=[1991, 2055], dtick=10),
    yaxis=dict(title='Flood Days', range=[0, max(station_proj_df[sea_level_rise_scenarios].max().max(), avg_flood_days_per_decade[obs_count_col].max()) * 1.1]),
    legend=dict(x=0, y=1.0, traceorder='normal', title='Legend'),
    barmode='overlay',
    bargap=0.1,
    bargroupgap=0.1,
    plot_bgcolor='rgba(255, 255, 255, 1)',
    paper_bgcolor='rgba(255, 255, 255, 1)',
    width=1200,
    height=600,
    title_x=0.5
)

fig.show()

## **Takeaways**

###Thank you for taking the time to walk through this Annual HTF Outlook Station Exploration notebook.

###Some key takeaways include:

*   How to access CO-OPS data from the CO-OPS API (here is a [URL builder](https://tidesandcurrents.noaa.gov/api-helper/url-generator.html) if you want to explore other CO-OPS data)
*   How Annual HTF predictions vary across the nation
*   How to calculate the relative change in HTF days since 2000
*   How ENSO can have different impacts on HTF prediction regionally
*   How CO-OPS predicts the next met-year using different fits with and without ENSO sensitivity.
*   Understanding and plotting how HTF days may change with different emission based scenarios in future decades.
